In [1]:
# 1. Import Libraries and Load Data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import time

# Load data
data = pd.read_csv(r'D:\william\OneDrive - UW-Madison\UW-Madison\722\-2025AAE722_William-J\Final Project\DataCoSupplyChainDataset1.csv')
print(f"Data loaded: {data.shape[0]} records, {data.shape[1]} columns")

Data loaded: 180519 records, 53 columns


In [2]:
# 2. Data Preprocessing and Feature Selection
# Select features including Customer Country
features = [
    'Type',  # Payment type
    'Category Name', 
    'Benefit per order',  # NEW: Added benefit per order
    'Market', 
    'Customer Country',  # NEW: Added customer country
    'Order Item Discount Rate', 
    'Order Item Product Price', 
    'Order Item Quantity',
    'Shipping Mode',
    'Late_delivery_risk',
    'Delivery Status',
    'Order Status',
    'Department Name',
    'order date (DateOrders)',
    'Order Item Profit Ratio'  # Target variable
]

# Create clean dataset
model_data = data[features].copy().dropna()
model_data['order date (DateOrders)'] = pd.to_datetime(model_data['order date (DateOrders)'])

# Create separate date and time columns
model_data['Order Date'] = model_data['order date (DateOrders)'].dt.date
model_data['Order Time'] = model_data['order date (DateOrders)'].dt.time
print(f"Clean data: {model_data.shape[0]} records")
print(f"Customer Country unique values: {model_data['Customer Country'].nunique()}")
print(f"Type unique values: {model_data['Type'].nunique()}")
print(f"Category Name unique values: {model_data['Category Name'].nunique()}")

Clean data: 180519 records
Customer Country unique values: 2
Type unique values: 4
Category Name unique values: 50


# Convert time vars

In [3]:
# 3a. Time Feature Engineering (Before Encoding)

# Extract meaningful time components from order date
model_data['order_month'] = model_data['order date (DateOrders)'].dt.month
model_data['order_quarter'] = model_data['order date (DateOrders)'].dt.quarter
model_data['order_day_of_week'] = model_data['order date (DateOrders)'].dt.dayofweek


# Business-relevant time features
model_data['is_weekend'] = (model_data['order_day_of_week'] >= 5).astype(int)
model_data['is_month_end'] = (model_data['order date (DateOrders)'].dt.day >= 28).astype(int)
model_data['is_quarter_end'] = model_data['order_month'].isin([3, 6, 9, 12]).astype(int)

# Seasonal categories
def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Fall'

model_data['season'] = model_data['order_month'].apply(get_season)
model_data


,Type,Category Name,Benefit per order,Market,Customer Country,Order Item Discount Rate,Order Item Product Price,Order Item Quantity,Shipping Mode,Late_delivery_risk,...,Order Item Profit Ratio,Order Date,Order Time,order_month,order_quarter,order_day_of_week,is_weekend,is_month_end,is_quarter_end,season
0,DEBIT,Sporting Goods,91.250000,Pacific Asia,Puerto Rico,0.04,327.750000,1,Standard Class,0,...,0.29,2018-01-31,22:56:00,1,1,2,0,1,0,Winter
1,TRANSFER,Sporting Goods,-249.089996,Pacific Asia,Puerto Rico,0.05,327.750000,1,Standard Class,1,...,-0.80,2018-01-13,12:27:00,1,1,5,1,0,0,Winter
2,CASH,Sporting Goods,-247.779999,Pacific Asia,EE. UU.,0.06,327.750000,1,Standard Class,0,...,-0.80,2018-01-13,12:06:00,1,1,5,1,0,0,Winter
3,DEBIT,Sporting Goods,22.860001,Pacific Asia,EE. UU.,0.07,327.750000,1,Standard Class,0,...,0.08,2018-01-13,11:45:00,1,1,5,1,0,0,Winter
4,PAYMENT,Sporting Goods,134.210007,Pacific Asia,Puerto Rico,0.09,327.750000,1,Standard Class,0,...,0.45,2018-01-13,11:24:00,1,1,5,1,0,0,Winter
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180514,CASH,Fishing,40.000000,Pacific Asia,EE. UU.,0.00,399.980011,1,Standard Class,0,...,0.10,2016-01-16,03:40:00,1,1,5,1,0,0,Winter
180515,DEBIT,Fishing,-613.770019,Pacific Asia,EE. UU.,0.01,399.980011,1,Second Class,1,...,-1.55,2016-01-16,01:34:00,1,1,5,1,0,0,Winter
180516,TRANSFER,Fishing,141.110001,Pacific Asia,EE. UU.,0.02,399.980011,1,Standard Class,1,...,0.36,2016-01-15,21:00:00,1,1,4,0,0,0,Winter
180517,PAYMENT,Fishing,186.229996,Pacific Asia,Puerto Rico,0.03,399.980011,1,Standard Class,0,...,0.48,2016-01-15,20:18:00,1,1,4,0,0,0,Winter


# Get dummy variables for categorical features

In [4]:
categorical_features = [
    'Type', 'Category Name', 'Market', 'Customer Country', 
    'Shipping Mode', 'Delivery Status', 'Order Status', 'Department Name',
    'season','order_month', 'order_quarter', 'order_day_of_week'
]
for feature in categorical_features:
    dummies = pd.get_dummies(model_data[feature], prefix=feature)
    model_data = pd.concat([model_data, dummies], axis=1)
model_data.head()

,Type,Category Name,Benefit per order,Market,Customer Country,Order Item Discount Rate,Order Item Product Price,Order Item Quantity,Shipping Mode,Late_delivery_risk,...,order_quarter_2,order_quarter_3,order_quarter_4,order_day_of_week_0,order_day_of_week_1,order_day_of_week_2,order_day_of_week_3,order_day_of_week_4,order_day_of_week_5,order_day_of_week_6
0,DEBIT,Sporting Goods,91.250000,Pacific Asia,Puerto Rico,0.04,327.75,1,Standard Class,0,...,False,False,False,False,False,True,False,False,False,False
1,TRANSFER,Sporting Goods,-249.089996,Pacific Asia,Puerto Rico,0.05,327.75,1,Standard Class,1,...,False,False,False,False,False,False,False,False,True,False
2,CASH,Sporting Goods,-247.779999,Pacific Asia,EE. UU.,0.06,327.75,1,Standard Class,0,...,False,False,False,False,False,False,False,False,True,False
3,DEBIT,Sporting Goods,22.860001,Pacific Asia,EE. UU.,0.07,327.75,1,Standard Class,0,...,False,False,False,False,False,False,False,False,True,False
4,PAYMENT,Sporting Goods,134.210007,Pacific Asia,Puerto Rico,0.09,327.75,1,Standard Class,0,...,False,False,False,False,False,False,False,False,True,False


# Time Series Analysis

In [9]:
target = 'Benefit per order'
y_ts = model_data[target]

X_Bts = model_data.drop(columns=[target, 'order date (DateOrders)','Order Date', 'Order Time',
    'Type', 'Category Name', 'Market', 'Customer Country', 
    'Shipping Mode', 'Delivery Status', 'Order Status', 'Department Name',
    'season','order_month', 'order_quarter', 'order_day_of_week','Benefit per order'
])

In [10]:
import statsmodels.api as sm

# Convert boolean columns to integers for statsmodels compatibility
X_numeric = X_Bts.astype(int)

Baseline_Regression = sm.OLS(y_ts, sm.add_constant(X_numeric)).fit()
Baseline_Regression.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      Benefit per order   R-squared:                       0.431
Model:                            OLS   Adj. R-squared:                  0.431
Method:                 Least Squares   F-statistic:                     1537.
Date:                Fri, 28 Nov 2025   Prob (F-statistic):               0.00
Time:                        10:42:10   Log-Likelihood:            -1.0444e+06
No. Observations:              180519   AIC:                         2.089e+06
Df Residuals:                  180429   BIC:                         2.090e+06
Df Model:                          89                                         
Covariance Type:            nonrobust                                         
======================================================================================================
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
const                                  5.8514      0.551     10.629      0.000       4.772       6.930
Order Item Discount Rate            5.328e-15   8.57e-15      0.622      0.534   -1.15e-14    2.21e-14
Order Item Product Price               0.0274      0.009      3.012      0.003       0.010       0.045
Order Item Quantity                    6.0429      0.173     34.850      0.000       5.703       6.383
Late_delivery_risk                     0.7939      0.190      4.185      0.000       0.422       1.166
Order Item Profit Ratio              291.8703      0.806    362.163      0.000     290.291     293.450
is_weekend                             1.1007      0.248      4.436      0.000       0.614       1.587
is_month_end                          -0.1201      0.590     -0.204      0.839      -1.277       1.036
is_quarter_end                         0.6300      0.299      2.104      0.035       0.043       1.217
Type_CASH                              1.4917      0.283      5.273      0.000       0.937       2.046
Type_DEBIT                             1.1419      0.309      3.700      0.000       0.537       1.747
Type_PAYMENT                           1.5526      0.503      3.086      0.002       0.567       2.539
Type_TRANSFER                          1.6653      0.320      5.203      0.000       1.038       2.293
Category Name_Accessories            -15.0247      2.529     -5.941      0.000     -19.981     -10.068
Category Name_As Seen on  TV!         -5.7970      9.309     -0.623      0.533     -24.041      12.448
Category Name_Baby                   -11.4934      5.093     -2.257      0.024     -21.476      -1.511
Category Name_Baseball & Softball     -4.3825      3.380     -1.297      0.195     -11.007       2.242
Category Name_Basketball               6.6226      8.834      0.750      0.453     -10.691      23.936
Category Name_Books                   -1.2810      2.014     -0.636      0.525      -5.228       2.666
Category Name_Boxing & MMA             0.4491      4.304      0.104      0.917      -7.986       8.884
Category Name_CDs                     -8.7438      4.173     -2.096      0.036     -16.922      -0.566
Category Name_Cameras                 -9.6010      3.103     -3.094      0.002     -15.683      -3.519
Category Name_Camping & Hiking        10.0606      1.503      6.692      0.000       7.114      13.007
Category Name_Cardio Equipment         9.5516      2.371      4.028      0.000       4.904      14.199
Category Name_Children's Clothing     11.7547      3.262      3.603      0.000       5.361      18.149
Category Name_Cleats                 -12.0094      1.909     -6.292      0.000     -15.750      -8.268
Category Name_Computers               71.1483      8.755      8.127      0.000      53.989      

In [11]:
# Lag Discount

LD_data = model_data.copy()


In [53]:
# Sort by date to ensure proper time series order
LD_data = LD_data.sort_values('Order Date')

# Create category-market combinations
LD_data['cat_market'] = LD_data['Category Name'] + '_' + LD_data['Market']

# Initialize lag discount columns
for lag_days in range(1, 7):  # 1-6 days
    LD_data[f'lag_discount_{lag_days}d'] = np.nan


# Group by category-market for efficient processing
grouped = LD_data.groupby('cat_market')

for lag_days in range(1, 7):
    print(f"Processing lag {lag_days} days...")
    
    def calculate_lag_discount(group):
        # Sort by date
        group = group.sort_values('Order Date')
        
        # For each row, find the most recent discount data within the lag window
        lag_discounts = []
        
        for idx, row in group.iterrows():
            current_date = row['Order Date']
            
            # Look for data in the lag window (from lag_days to lag_days+2 to allow some flexibility)
            min_lag_date = current_date - pd.Timedelta(days=lag_days+2)
            max_lag_date = current_date - pd.Timedelta(days=max(1, lag_days-1))
            
            # Get historical data within the lag window, excluding current date
            historical_mask = (
                (group['Order Date'] >= min_lag_date) & 
                (group['Order Date'] <= max_lag_date) &
                (group['Order Date'] < current_date)
            )
            historical_data = group[historical_mask]
            
            if len(historical_data) > 0:
                # Get the most recent data within the window
                most_recent_data = historical_data.loc[historical_data['Order Date'].idxmax()]
                avg_discount = historical_data['Order Item Discount Rate'].mean()
            else:
                avg_discount = np.nan
            
            lag_discounts.append(avg_discount)
        
        # Return series with original index
        return pd.Series(lag_discounts, index=group.index)
    
    # Apply function to each group and store results
    lag_column = f'lag_discount_{lag_days}d'
    LD_data[lag_column] = grouped.apply(calculate_lag_discount).reset_index(level=0, drop=True)

print("Lag discount calculation completed!")

# Fill NaN values with the closest earlier non-NaN value from same cat_market
for lag_days in range(1, 7):
    col_name = f'lag_discount_{lag_days}d'
    
    # Sort by date to ensure proper forward fill order
    LD_data = LD_data.sort_values(['cat_market', 'Order Date'])
    
    # Forward fill within each category-market group
    LD_data[col_name] = LD_data.groupby('cat_market')[col_name].fillna(method='ffill')
    
    # If still NaN (no earlier values), fill with category-market mean as fallback
    cat_market_means = LD_data.groupby('cat_market')['Order Item Discount Rate'].mean()
    for combo in LD_data['cat_market'].unique():
        mask = (LD_data['cat_market'] == combo) & (LD_data[col_name].isna())
        if mask.sum() > 0:
            LD_data.loc[mask, col_name] = cat_market_means[combo]

LD_data


Processing lag 1 days...
Processing lag 2 days...
Processing lag 2 days...
Processing lag 3 days...
Processing lag 3 days...
Processing lag 4 days...
Processing lag 4 days...
Processing lag 5 days...
Processing lag 5 days...
Processing lag 6 days...
Processing lag 6 days...
Lag discount calculation completed!
Lag discount calculation completed!


,Type,Category Name,Benefit per order,Market,Customer Country,Order Item Discount Rate,Order Item Product Price,Order Item Quantity,Shipping Mode,Late_delivery_risk,...,order_day_of_week_5,order_day_of_week_6,cat_market,lag_discount_1d,lag_discount_2d,lag_discount_3d,lag_discount_4d,lag_discount_5d,lag_discount_6d,Order_Date_dt
43591,DEBIT,Accessories,25.709999,Africa,EE. UU.,0.02,24.990000,5,First Class,1,...,False,False,Accessories_Africa,0.104507,0.104507,0.104507,0.104507,0.104507,0.104507,2016-08-26
84504,TRANSFER,Accessories,-4.090000,Africa,EE. UU.,0.13,24.990000,4,Second Class,1,...,False,True,Accessories_Africa,0.020000,0.020000,0.020000,0.104507,0.104507,0.104507,2016-08-28
48243,DEBIT,Accessories,-16.389999,Africa,EE. UU.,0.18,24.990000,1,Second Class,0,...,False,True,Accessories_Africa,0.020000,0.020000,0.020000,0.104507,0.104507,0.104507,2016-08-28
19426,PAYMENT,Accessories,5.540000,Africa,EE. UU.,0.12,24.990000,4,First Class,1,...,False,True,Accessories_Africa,0.020000,0.020000,0.020000,0.104507,0.104507,0.104507,2016-08-28
117544,TRANSFER,Accessories,16.180000,Africa,EE. UU.,0.17,24.990000,3,Same Day,1,...,False,False,Accessories_Africa,0.112500,0.112500,0.020000,0.020000,0.104507,0.104507,2016-08-29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51838,TRANSFER,Women's Golf Clubs,-56.580002,LATAM,EE. UU.,0.15,209.990005,1,Standard Class,0,...,False,False,Women's Golf Clubs_LATAM,0.050000,0.100000,0.140000,0.110000,0.110000,0.118571,2017-06-12
51773,TRANSFER,Women's Golf Clubs,178.330002,LATAM,EE. UU.,0.13,99.989998,5,Second Class,1,...,False,False,Women's Golf Clubs_LATAM,0.123333,0.123333,0.100000,0.140000,0.110000,0.110000,2017-06-13
51716,CASH,Women's Golf Clubs,74.080002,LATAM,EE. UU.,0.06,79.989998,2,First Class,1,...,False,False,Women's Golf Clubs_LATAM,0.125000,0.125000,0.123333,0.100000,0.140000,0.110000,2017-06-14
51772,PAYMENT,Women's Golf Clubs,68.050003,LATAM,Puerto Rico,0.17,99.989998,2,Standard Class,1,...,True,False,Women's Golf Clubs_LATAM,0.060000,0.095000,0.127500,0.112000,0.125000,0.123333,2017-06-17


In [54]:
# Debug: Check data structure and lag calculation
print("Debug Information:")
print(f"LD_data shape: {LD_data.shape}")
print(f"Date range: {LD_data['Order Date'].min()} to {LD_data['Order Date'].max()}")
print(f"Unique dates: {LD_data['Order Date'].nunique()}")
print(f"Unique cat_market combinations: {LD_data['cat_market'].nunique()}")

# Check sample of discount values
print("\nSample of Order Item Discount Rate:")
print(LD_data['Order Item Discount Rate'].describe())

# Check first few rows of lag features
lag_cols = [col for col in LD_data.columns if col.startswith('lag_discount_')]
print("\nFirst 10 rows of lag discount features:")
print(LD_data[['Order Date', 'cat_market', 'Order Item Discount Rate'] + lag_cols].head(10))

# Check for specific category-market combination
sample_combo = LD_data['cat_market'].iloc[0]
sample_data = LD_data[LD_data['cat_market'] == sample_combo].head(10)
print(f"\nSample data for {sample_combo}:")
print(sample_data[['Order Date', 'Order Item Discount Rate'] + lag_cols])

Debug Information:
LD_data shape: (180519, 148)
Date range: 2015-01-01 to 2018-01-31
Unique dates: 1127
Unique cat_market combinations: 162

Sample of Order Item Discount Rate:
count    180519.000000
mean          0.101668
std           0.070415
min           0.000000
25%           0.040000
50%           0.100000
75%           0.160000
max           0.250000
Name: Order Item Discount Rate, dtype: float64

First 10 rows of lag discount features:
        Order Date          cat_market  Order Item Discount Rate  \
43591   2016-08-26  Accessories_Africa                      0.02   
84504   2016-08-28  Accessories_Africa                      0.13   
48243   2016-08-28  Accessories_Africa                      0.18   
19426   2016-08-28  Accessories_Africa                      0.12   
117544  2016-08-29  Accessories_Africa                      0.17   
129573  2016-08-30  Accessories_Africa                      0.02   
109011  2016-08-30  Accessories_Africa                      0.01   
50205  

In [52]:
# Check date distribution for a specific category-market
sample_combo = LD_data['cat_market'].iloc[0]
sample_data = LD_data[LD_data['cat_market'] == sample_combo].sort_values('Order Date')

print(f"Date distribution for {sample_combo}:")
print("First 20 dates:")
print(sample_data['Order Date'].head(20).tolist())

# Check if dates are consecutive
dates = sample_data['Order Date'].unique()
dates_sorted = sorted(dates)
print(f"\nTotal unique dates for this combo: {len(dates_sorted)}")
if len(dates_sorted) > 1:
    print("First few date gaps:")
    for i in range(min(10, len(dates_sorted)-1)):
        gap = (dates_sorted[i+1] - dates_sorted[i]).days
        print(f"  {dates_sorted[i]} to {dates_sorted[i+1]}: {gap} days gap")

# Check discount values distribution
print(f"\nDiscount rate distribution for {sample_combo}:")
print(sample_data['Order Item Discount Rate'].describe())

Date distribution for Accessories_Africa:
First 20 dates:
[datetime.date(2016, 8, 26), datetime.date(2016, 8, 28), datetime.date(2016, 8, 28), datetime.date(2016, 8, 28), datetime.date(2016, 8, 29), datetime.date(2016, 8, 30), datetime.date(2016, 8, 30), datetime.date(2016, 8, 30), datetime.date(2016, 9, 1), datetime.date(2016, 9, 2), datetime.date(2016, 9, 2), datetime.date(2016, 9, 2), datetime.date(2016, 9, 3), datetime.date(2016, 9, 4), datetime.date(2016, 9, 4), datetime.date(2016, 9, 5), datetime.date(2016, 9, 9), datetime.date(2016, 9, 9), datetime.date(2016, 9, 10), datetime.date(2016, 9, 12)]

Total unique dates for this combo: 89
First few date gaps:
  2016-08-26 to 2016-08-28: 2 days gap
  2016-08-28 to 2016-08-29: 1 days gap
  2016-08-29 to 2016-08-30: 1 days gap
  2016-08-30 to 2016-09-01: 2 days gap
  2016-09-01 to 2016-09-02: 1 days gap
  2016-09-02 to 2016-09-03: 1 days gap
  2016-09-03 to 2016-09-04: 1 days gap
  2016-09-04 to 2016-09-05: 1 days gap
  2016-09-05 to 201

In [55]:
# Check updated lag features
lag_cols = [col for col in LD_data.columns if col.startswith('lag_discount_')]
print("Updated Lag Features Statistics:")
print(LD_data[lag_cols].describe())

print("\nNaN counts in lag features:")
for col in lag_cols:
    nan_count = LD_data[col].isna().sum()
    total_count = len(LD_data)
    print(f"{col}: {nan_count} NaNs ({100*nan_count/total_count:.1f}%)")

print("\nSample of lag features (first 10 rows):")
print(LD_data[['Order Date', 'cat_market', 'Order Item Discount Rate'] + lag_cols].head(10))

Updated Lag Features Statistics:
       lag_discount_1d  lag_discount_2d  lag_discount_3d  lag_discount_4d  \
count    180519.000000    180519.000000    180519.000000    180519.000000   
mean          0.101572         0.101582         0.101552         0.101543   
std           0.017858         0.015540         0.015471         0.015547   
min           0.000000         0.000000         0.000000         0.000000   
25%           0.096170         0.097841         0.097766         0.097838   
50%           0.101667         0.101667         0.101667         0.101667   
75%           0.107000         0.105347         0.105294         0.105284   
max           0.250000         0.250000         0.250000         0.250000   

       lag_discount_5d  lag_discount_6d  
count    180519.000000    180519.000000  
mean          0.101623         0.101559  
std           0.015736         0.015448  
min           0.000000         0.000000  
25%           0.097874         0.097927  
50%           0.10166

In [75]:
target = 'Order Item Quantity'
y_ts = LD_data[target]
#'Benefit per order'
X_ts = LD_data[['Order Item Discount Rate','Order Item Product Price','is_weekend','lag_discount_1d',
                #'lag_discount_2d','lag_discount_3d','lag_discount_4d','lag_discount_5d','lag_discount_6d'
                ]]

In [76]:
# Fill NaN values in lag discount columns before converting to int
lag_columns = [col for col in X_ts.columns if col.startswith('lag_discount_')]
for col in lag_columns:
	X_ts[col] = X_ts[col].fillna(X_ts[col].mean())  # Fill NaN with 0 or use another strategy
      
X_ts = X_ts.astype(int)

TS_Regression = sm.OLS(y_ts, sm.add_constant(X_ts)).fit()
TS_Regression.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                             OLS Regression Results                            
===============================================================================
Dep. Variable:     Order Item Quantity   R-squared:                       0.227
Model:                             OLS   Adj. R-squared:                  0.227
Method:                  Least Squares   F-statistic:                 2.644e+04
Date:                 Fri, 28 Nov 2025   Prob (F-statistic):               0.00
Time:                         15:00:53   Log-Likelihood:            -3.0046e+05
No. Observations:               180519   AIC:                         6.009e+05
Df Residuals:                   180516   BIC:                         6.010e+05
Df Model:                            2                                         
Covariance Type:             nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                        2.8191      0.005    604.564      0.000       2.810       2.828
Order Item Discount Rate  2.509e-14   4.19e-17    599.036      0.000     2.5e-14    2.52e-14
Order Item Product Price    -0.0050   2.15e-05   -229.960      0.000      -0.005      -0.005
is_weekend                   0.0138      0.007      2.076      0.038       0.001       0.027
lag_discount_1d                   0          0        nan        nan           0           0
==============================================================================
Omnibus:                    15892.663   Durbin-Watson:                   1.373
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            20470.633
Skew:                           0.820   Prob(JB):                         0.00
Kurtosis:                       3.177   Cond. No.                          inf
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is      0. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

# Lag Discount Impact Regression Analysis

In [61]:
# Focus regression on lag discount features only
print("="*60)
print("LAG DISCOUNT IMPACT REGRESSION ANALYSIS")
print("="*60)

# Create dataset with only lag features and relevant controls
lag_columns = [col for col in X_ts.columns if col.startswith('lag_discount_')]
control_features = ['Order Item Discount Rate', 'Order Item Product Price', 'Order Item Quantity', 'Late_delivery_risk']

# Select features for lag impact analysis
X_lag = X_ts[lag_columns + control_features].copy()
y_lag = y_ts.copy()

print(f"Lag features included: {lag_columns}")
print(f"Control features: {control_features}")
print(f"Dataset shape: {X_lag.shape}")

# Convert to numeric and handle any remaining NaN values
X_lag_numeric = X_lag.astype(float)
X_lag_numeric = X_lag_numeric.fillna(X_lag_numeric.mean())

# Run regression with lag features
lag_regression = sm.OLS(y_lag, sm.add_constant(X_lag_numeric)).fit()

print("\n" + "="*60)
print("REGRESSION RESULTS: LAG DISCOUNT IMPACT")
print("="*60)
print(lag_regression.summary())

LAG DISCOUNT IMPACT REGRESSION ANALYSIS
Lag features included: ['lag_discount_1d', 'lag_discount_2d', 'lag_discount_3d', 'lag_discount_4d', 'lag_discount_5d', 'lag_discount_6d']
Control features: ['Order Item Discount Rate', 'Order Item Product Price', 'Order Item Quantity', 'Late_delivery_risk']
Dataset shape: (180519, 10)

REGRESSION RESULTS: LAG DISCOUNT IMPACT
                            OLS Regression Results                            
Dep. Variable:      Benefit per order   R-squared:                       0.016
Model:                            OLS   Adj. R-squared:                  0.016
Method:                 Least Squares   F-statistic:                     989.1
Date:                Fri, 28 Nov 2025   Prob (F-statistic):               0.00
Time:                        14:52:27   Log-Likelihood:            -1.0938e+06
No. Observations:              180519   AIC:                         2.188e+06
Df Residuals:                  180515   BIC:                         2.188e+06
D

# RF Model Hyperparameter Tuning

In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.ensemble import VotingRegressor
import time


In [ ]:
# 3b. Updated Feature Definition with Time Components
# Separate target and features
target = 'Order Item Profit Ratio'
y = model_data[target]

X = model_data.drop(columns=[target, 'order date (DateOrders)','Order Date', 'Order Time',
    'Type', 'Category Name', 'Market', 'Customer Country', 
    'Shipping Mode', 'Delivery Status', 'Order Status', 'Department Name',
    'season','order_month', 'order_quarter', 'order_day_of_week','Benefit per order'
])

In [41]:
X

,Order Item Discount Rate,Order Item Product Price,Order Item Quantity,Late_delivery_risk,is_weekend,is_month_end,is_quarter_end,Type_CASH,Type_DEBIT,Type_PAYMENT,...,order_quarter_2,order_quarter_3,order_quarter_4,order_day_of_week_0,order_day_of_week_1,order_day_of_week_2,order_day_of_week_3,order_day_of_week_4,order_day_of_week_5,order_day_of_week_6
0,0.04,327.750000,1,0,0,1,0,False,True,False,...,False,False,False,False,False,True,False,False,False,False
1,0.05,327.750000,1,1,1,0,0,False,False,False,...,False,False,False,False,False,False,False,False,True,False
2,0.06,327.750000,1,0,1,0,0,True,False,False,...,False,False,False,False,False,False,False,False,True,False
3,0.07,327.750000,1,0,1,0,0,False,True,False,...,False,False,False,False,False,False,False,False,True,False
4,0.09,327.750000,1,0,1,0,0,False,False,True,...,False,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180514,0.00,399.980011,1,0,1,0,0,True,False,False,...,False,False,False,False,False,False,False,False,True,False
180515,0.01,399.980011,1,1,1,0,0,False,True,False,...,False,False,False,False,False,False,False,False,True,False
180516,0.02,399.980011,1,1,0,0,0,False,False,False,...,False,False,False,False,False,False,False,True,False,False
180517,0.03,399.980011,1,0,0,0,0,False,False,True,...,False,False,False,False,False,False,False,True,False,False


In [42]:
# 4. Train-Test Split (70% / 30%)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

print(f"Training: {X_train.shape[0]} samples")
print(f"Testing: {X_test.shape[0]} samples")

Training: 126363 samples
Testing: 54156 samples


In [34]:
# 5. Hybrid Model Implementation with Cross-Validation

# Define individual models with initial parameters
rf_model = RandomForestRegressor(random_state=42)

# Hyperparameter grids for optimization
param_grids = {
    'Random Forest': {
        'n_estimators': [1000],
        'max_depth': [5,6,7,8,9,10],
        'min_samples_split': [5,6,7,8,9,10],
        'min_samples_leaf': [1,2,3,4]
    },
}

# Cross-validation results storage
cv_results = {}
optimized_models = {}
best_params = {}

# Run GridSearchCV for Random Forest
sample_size = int(0.1 * len(X_train))
X_sample = X_train.sample(n=sample_size, random_state=42)
y_sample = y_train.loc[X_sample.index]

rf_grid = GridSearchCV(
    estimator=rf_model,
    param_grid=param_grids['Random Forest'],
    cv=3,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=0
)

rf_grid.fit(X_sample, y_sample)

# Store results
best_params['Random Forest'] = rf_grid.best_params_
optimized_models['Random Forest'] = rf_grid.best_estimator_
cv_results['Random Forest'] = -rf_grid.best_score_

print(f"Best RF Parameters: {rf_grid.best_params_}")



Best RF Parameters: {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 1000}


# RF Model

In [43]:
# Train optimal RF model
optimal_rf = optimized_models['Random Forest']
optimal_rf.fit(X_train, y_train)

# Test performance
rf_test_pred = optimal_rf.predict(X_test)
rf_test_MSE = mean_squared_error(y_test, rf_test_pred)

print(f"Test MSE: {rf_test_MSE:.4f}")

# Feature importance
feature_importance = pd.DataFrame({
    'Feature': X.columns,
    'Importance': optimal_rf.feature_importances_
}).sort_values('Importance', ascending=False)

print("\nFeature Importance:")
for i, row in feature_importance.iterrows():
    print(f"{row['Feature']:30s} {row['Importance']:.4f}")

Test MSE: 0.2146

Feature Importance:
Order Item Discount Rate       0.1540
Order Item Quantity            0.0428
Order Item Product Price       0.0428
is_month_end                   0.0205
order_day_of_week_1            0.0177
Shipping Mode_Same Day         0.0169
Delivery Status_Shipping on time 0.0167
Shipping Mode_Second Class     0.0165
order_day_of_week_2            0.0163
order_day_of_week_3            0.0163
Order Status_PENDING           0.0162
Shipping Mode_First Class      0.0156
order_day_of_week_0            0.0154
Order Status_PROCESSING        0.0148
Delivery Status_Advance shipping 0.0147
Order Status_ON_HOLD           0.0147
Shipping Mode_Standard Class   0.0144
order_day_of_week_4            0.0141
order_day_of_week_5            0.0141
order_day_of_week_6            0.0139
Customer Country_Puerto Rico   0.0135
Order Status_PENDING_PAYMENT   0.0131
Customer Country_EE. UU.       0.0127
Order Status_COMPLETE          0.0123
Market_Africa                  0.0107
Market_P